## 종속성 설치

In [2]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade--quiet tokenizers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade--quiet


In [4]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.1 MB/s eta 0:00:00


## LLM 모델 준비

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 ID 설정
base_model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# 2. BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, # 4bit quantization으로 모델을 불러와 메모리 소모를 줄임
)

# 3. 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [6]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,  # 최대 토큰 길이
    padding_side="left",   # 입력 패딩 방향
    add_eos_token=True     # EOS 토큰 추가
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# LLM 파이프라인 설정 (LangChain과 통합)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-7-0e279de07470>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


## 데이터 준비

<br>
Langchain csvLoader를 사용해 문서를 로드.

In [8]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/서지.csv'

In [10]:
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/실라버스.csv'

In [11]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# CSV 파일 로드
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/서지.csv'
data = pd.read_csv(file_path)

# Replace NaN values in '책 소개' with an empty string
data['책 소개'] = data['책 소개'].fillna('')

# CSV 데이터를 LangChain Document로 변환
# Change metadata keys to start with an underscore and use English or abbreviations
bibliography_docs = [
    Document(
        page_content=row['책 소개'],  # 책 소개를 주요 내용으로 설정
        metadata={
            "title": row['제목'],       # Changed to 'title'
            "author": row['저자'],       # Changed to 'author'
            "publisher": row['출판사'],   # Changed to 'publisher'
            "isbn": row['ISBN']       # Changed to 'isbn'
        }
    )
    for _, row in data.iterrows()
]

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs1 = text_splitter.split_documents(bibliography_docs)

# 첫 번째 분할된 문서 출력
print(docs1[1])

page_content='<전염병의 세계사>와 <전쟁의 세계사>로 국내 독자들에게 알려진 바 있으며, 현존하는 가장 탁월한 역사가 중 한명으로 꼽히는 윌리엄 맥닐 시카고대 역사학과 교수의 책. 해외에서입니다.' metadata={'title': '세계의 역사', 'author': '윌리엄 맥닐', 'publisher': '이산', 'isbn': 271590}


In [12]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# CSV 파일 로드
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/실라버스.csv'
data = pd.read_csv(file_path)

# Replace NaN values in '책 소개' with an empty string
data['요약 설명'] = data['요약 설명'].fillna('')

# CSV 데이터를 LangChain Document로 변환
# Change metadata keys to start with an underscore and use English or abbreviations
syllabus_docs = [
    Document( # Create Document objects instead of dictionaries
        page_content=row['요약 설명'],  # 요약 설명을 주요 내용으로 설정
        metadata={
            "title": row['강의 명'],        # Changed to '_title'
            "course_id": row['강의 번호'],  # Changed to '_course_id'
            "department": row['주관 학과'], # Changed to '_department'
            "reference": row['참고문헌']   # Changed to '_reference'
        }
    )
    for _, row in data.iterrows()
]

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs2 = text_splitter.split_documents(syllabus_docs)

# 첫 번째 분할된 문서 출력
print(docs2[22])

page_content='인적자원 관리와 조직의 동기부여 및 성과 평가 기법을 탐구합니다.' metadata={'title': '인적자원관리', 'course_id': 1022, 'department': '경영학', 'reference': 'Human Resource Management'}


## 임베딩 모델 활용 데이터 임베딩 이후, Milvus 벡터 DB에 저장

<br>

문서로 Milvus 벡터 스토어를 초기화하여 Milvus 벡터 스토어에 문서를 로드하고 내부적으로 인덱스를 구축.

In [13]:
!pip install pymilvus

In [14]:
from langchain_milvus import Milvus
from langchain.embeddings import HuggingFaceEmbeddings

#  Milvus 컬렉션 생성
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': 'cuda:0'})

# 서지 컬렉션
bibliography_store = Milvus.from_documents(
    documents=docs1,
    embedding=embedding_model,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    collection_name="bibliography_collection",
    drop_old=True,
    # Specify index_params for FLAT index
    index_params={"index_type": "FLAT"}
)

# 실라버스 컬렉션
syllabus_store = Milvus.from_documents(
    documents=docs2,
    embedding=embedding_model,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    collection_name="syllabus_collection",
    drop_old=True,
    # Specify index_params for FLAT index
    index_params={"index_type": "FLAT"}
)

<ipython-input-14-55cbef13c5ba>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': 'cuda:0'})


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 서지 컬렉션 검색
테스트 쿼리 문제를 사용하여 Milvus 벡터 스토어에서 문서를 검색합니다. 상위 2개 문서를 살펴보기.

In [33]:
query = "거시경제학"
results1 = bibliography_store.similarity_search(query, k=2)

In [34]:
results1

[Document(metadata={'author': '정운찬, 김영식', 'isbn': 599922, 'pk': 455456081957093483, 'publisher': '율곡출판사', 'title': '거시경제론'}, page_content='이 책은 거시경제론을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.'),
 Document(metadata={'author': 'Andrew B. Abel, Ben S. Bernanke, Dean Croushore', 'isbn': 514313, 'pk': 455456081957093481, 'publisher': '지필미디어', 'title': '거시경제학'}, page_content='『거시경제학』은 경제학을 공부하는 학생들을 위해 고전학파와 케인즈파의 여러 가정을 명료하게 설명하는 단일 경제모형을 바탕으로 하나로 통일된 접근방법을 제시한 책이다. 특히 세계입니다.')]

### 실라버스 컬렉션 검색
테스트 쿼리 문제를 사용하여 Milvus 벡터 스토어에서 문서를 검색합니다. 상위 2개 문서를 살펴보기.

In [35]:
query = "거시경제학"
results2 = syllabus_store.similarity_search(query, k=2)

In [36]:
results2

[Document(metadata={'course_id': 1041, 'department': '경제학', 'pk': 455456082151080677, 'reference': 'Macroeconomics', 'title': '거시경제학'}, page_content='국가 경제와 정책의 영향을 다루는 거시경제학 원리를 탐구합니다.'),
 Document(metadata={'course_id': 1045, 'department': '경제학', 'pk': 455456082151080681, 'reference': 'Financial Economics', 'title': '금융 경제학'}, page_content='금융 시장의 구조와 경제학적 원리를 탐구합니다.')]

## RAG 체인 구축하기
Langchain 기반 RAG 체인을 구축합니다.

In [72]:
# LangChain Prompt 설정
PROMPT_TEMPLATE = """
당신은 서지 및 강의 정보를 분석하여 추천하는 AI 비서입니다.
아래 제공된 정보(context)와 <question> 태그 안에 있는 사용자 질문을 기반으로 **지정된 응답 형식만** 따르세요.
하나, 질문에 대해 알맞은 도서를 추천하고 이유를 제시하라.
둘, 추천 도서에 대해 학습 단계별 로드맵을 작성하고, 각 단계에 대한 이유를 제시하라.
셋, 질문에 대해 알맞은 강의를 추천하고 이유를 제시하라.
넷, 응답 형식을 벗어난 내용은 절대 작성하지 마세요.

<정보>
{context}
</정보>

<질문>
{question}
</질문>

## 지정 응답 형식:
1. 추천 도서 목록:
- [도서명] (저자)
  이유: [추천 이유]

2. 도서 로드맵:
- 단계 1: [도서명] (저자)
  이유: [이 도서를 먼저 읽어야 하는 이유]
- 단계 2: [도서명] (저자)
  이유: [이 도서를 다음 단계로 추천하는 이유]

3. 추천 강의 목록:
- [강의명]
  이유: [추천 이유]

반드시 위 지정 응답 형식만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가, 반복 생성 없이 종료하세요.
========================= 실제 생성 ===========================================

"""


In [73]:
from langchain_core.prompts import PromptTemplate
# PromptTemplate 설정
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

In [74]:
# 두 개의 컬렉션에서 데이터를 검색하는 함수
def search_collections(query, syllabus_store, bibliography_store, top_k=2):
    query_embedding = embedding_model.embed_query(query)

    # 실라버스 컬렉션에서 검색
    syllabus_results = syllabus_store.similarity_search_by_vector(query_embedding, k=top_k)
    syllabus_context = "\n".join(
        f"강의명: {res.metadata['title']}, 강의 번호: {res.metadata['course_id']}, 주관 학과: {res.metadata['department']}\n{res.page_content}"
        for res in syllabus_results
    )

    # 서지 컬렉션에서 검색
    bibliography_results = bibliography_store.similarity_search_by_vector(query_embedding, k=top_k)
    bibliography_context = "\n".join(
        f"도서명: {res.metadata['title']}, 저자: {res.metadata['author']}, 출판사: {res.metadata['publisher']}, ISBN: {res.metadata['isbn']}\n{res.page_content}"
        for res in bibliography_results
    )

    # 두 결과를 합침
    combined_context = f"실라버스 정보:\n{syllabus_context}\n\n서지 정보:\n{bibliography_context}"
    return combined_context

In [75]:
# 사용자 질의 처리 함수
def handle_query(query, syllabus_store, bibliography_store):
    # 검색 결과 가져오기
    context = search_collections(query, syllabus_store, bibliography_store)

    # Prompt에 맞게 데이터를 전달
    formatted_prompt = prompt.format(context=context, question=query)

    #LLM을 통해 결과 생성
    response = llm(formatted_prompt)

    # Check if the response is a string and convert to a dictionary if necessary
    if isinstance(response, str):
        response = {"generated_text": response}

    return response

In [76]:
# 사용자 질의 테스트
query = "거시경제와 미시경제의 차이를 알 수 있는 강의 및 도서를 추천해줘."
response = handle_query(query, syllabus_store, bibliography_store)

print("추천 결과:")
print(response["generated_text"]) # Access the 'generated_text' key directly

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


추천 결과:

당신은 서지 및 강의 정보를 분석하여 추천하는 AI 비서입니다.
아래 제공된 정보(context)와 <question> 태그 안에 있는 사용자 질문을 기반으로 **지정된 응답 형식만** 따르세요.
하나, 질문에 대해 알맞은 도서를 추천하고 이유를 제시하라.
둘, 추천 도서에 대해 학습 단계별 로드맵을 작성하고, 각 단계에 대한 이유를 제시하라.
셋, 질문에 대해 알맞은 강의를 추천하고 이유를 제시하라.
넷, 응답 형식을 벗어난 내용은 절대 작성하지 마세요.

<정보>
실라버스 정보:
강의명: 거시경제학, 강의 번호: 1041, 주관 학과: 경제학
국가 경제와 정책의 영향을 다루는 거시경제학 원리를 탐구합니다.
강의명: 미시경제학, 강의 번호: 1040, 주관 학과: 경제학
소비자의 선택과 기업의 생산 결정을 분석하는 미시경제학 이론을 학습합니다.

서지 정보:
도서명: 거시경제론, 저자: 정운찬, 김영식, 출판사: 율곡출판사, ISBN: 599922
이 책은 거시경제론을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.
도서명: Economic Policy, 저자: Ava, 출판사: Publisher23, ISBN: 411480
Economic Policy은 실라버스에 포함된 참고문헌으로 학습 자료로 사용됩니다.
</정보>

<질문>
거시경제와 미시경제의 차이를 알 수 있는 강의 및 도서를 추천해줘.
</질문>

## 지정 응답 형식:
1. 추천 도서 목록:
- [도서명] (저자)
  이유: [추천 이유]

2. 도서 로드맵:
- 단계 1: [도서명] (저자)
  이유: [이 도서를 먼저 읽어야 하는 이유]
- 단계 2: [도서명] (저자)
  이유: [이 도서를 다음 단계로 추천하는 이유]

3. 추천 강의 목록:
- [강의명]
  이유: [추천 이유]

반드시 위 지정 응답 형식만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가,